Imports

In [1]:
import csv
from PIL import Image
import cv2
import numpy as np
from sklearn.utils import shuffle as sk_shuffle
from keras.models import Sequential
from keras.layers import Lambda, Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Conv2D

print("Imports complete")

Using TensorFlow backend.


Imports complete


In [2]:
# Parameters
cropped_pixels = 50 #crop out sections of the image above this pixel positon on the y-axis
new_rows  = 11 #number of rows in the resized image
new_cols  = 32 #number of columns in the resized image
steering_correction = 0.3 #amount of steering wheel angle correction applied to images from the left and the right cameras
batch_size = 128

Pre-process Training Data

In [3]:
def pre_process(line, idx, new_cols, new_rows):
    # pre-process each incoming image
    source_path = line[idx]
    filename = source_path.split('/')[-1]
    current_path = 'data/IMG/' + filename
    img = Image.open(current_path)
    img = np.array(img)
    
    img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)[cropped_pixels:,:,1]  
    return cv2.resize(img, (new_cols, new_rows))

In [4]:
# load training data from the .csv file
lines = []
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

# Pop the first item on the list to get rid of list headers
lines.pop(0)

['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']

In [5]:
# Load X and y values from the training data

X = []
y = []

print("Loading data...")

for line in lines:
    # central camera
    X.append(pre_process(line, 0, new_cols, new_rows))
    y.append(float(line[3]))

    # left camera
    X.append(pre_process(line, 1, new_cols, new_rows))
    y.append(float(line[3]) + steering_correction)

    # right camera
    X.append(pre_process(line, 2, new_cols, new_rows))
    y.append(float(line[3]) - steering_correction)

X = np.array(X)
y = np.array(y)

print("Training Data Size: ", X.shape, y.shape)
print("Loading data complete")

Loading data...
Training Data Size:  (26463, 11, 32) (26463,)
Loading data complete


Data Augmentation

In [6]:
# flip the image data horizontally and append the new data to the training data
X = np.concatenate([X, X[:,:,::-1]])
# negate the steering wheel angles of the flipped images
y = np.concatenate([y, -y])

# shuffle the training data
X, y = sk_shuffle(X, y)

X_train = X[:,:,:,None]
y_train = y[:]

print("Training Data Size: ", X_train.shape, y_train.shape)
print("Loading data complete")
print("Flipping and shuffling of data complete")

Training Data Size:  (52926, 11, 32, 1) (52926,)
Loading data complete
Flipping and shuffling of data complete


Convolutional Neural Network Model Architecture

In [10]:
# Model design
model = Sequential()
model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(new_rows, new_cols, 1), name='Normalization'))
model.add(Conv2D(2, 1, 1, border_mode='same', activation='relu'))
model.add(MaxPooling2D((4, 4), (4, 4), 'same'))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(1))

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
Normalization (Lambda)           (None, 11, 32, 1)     0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 11, 32, 2)     4           Normalization[0][0]              
____________________________________________________________________________________________________
maxpooling2d_2 (MaxPooling2D)    (None, 3, 8, 2)       0           convolution2d_2[0][0]            
____________________________________________________________________________________________________
dropout_2 (Dropout)              (None, 3, 8, 2)       0           maxpooling2d_2[0][0]             
___________________________________________________________________________________________

Training the Network

In [11]:
from keras.callbacks import EarlyStopping

# Compile the model
model.compile(loss='mean_squared_error',optimizer='adam')

# Train the model on the training data with 25 epochs
model.fit(X_train, y_train, batch_size=batch_size, verbose=1, validation_split=0.1, nb_epoch=25,shuffle=True)

Train on 47633 samples, validate on 5293 samples
Epoch 1/25
47633/47633 [==============================] - 4s - loss: 0.2630 - val_loss: 0.0720
Epoch 2/25
47633/47633 [==============================] - 3s - loss: 0.0850 - val_loss: 0.0630
Epoch 3/25
47633/47633 [==============================] - 3s - loss: 0.0696 - val_loss: 0.0587
Epoch 4/25
47633/47633 [==============================] - 3s - loss: 0.0657 - val_loss: 0.0559
Epoch 5/25
47633/47633 [==============================] - 3s - loss: 0.0623 - val_loss: 0.0527
Epoch 6/25
47633/47633 [==============================] - 3s - loss: 0.0576 - val_loss: 0.0481
Epoch 7/25
47633/47633 [==============================] - 3s - loss: 0.0515 - val_loss: 0.0426
Epoch 8/25
47633/47633 [==============================] - 3s - loss: 0.0460 - val_loss: 0.0379
Epoch 9/25
47633/47633 [==============================] - 3s - loss: 0.0420 - val_loss: 0.0343
Epoch 10/25
47633/47633 [==============================] - 3s - loss: 0.0396 - val_loss: 0.0319


Save the Model

In [9]:
# for the new drive.py file
from keras.models import model_from_json

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save("model.h5")
print("Saved model to disk")

Saved model to disk
